# Cleaning loan repayments data

## Data Cleaning Actions to Take

1. Create dataframe with proper datatypes, and proper column names

2. Insert a new column named as ingestion date (current time )

3. Drop any nulls

4. Check if total_payment_received is zero, and total_principal_received is not zero. If there is such row then replace the value with sum of total_principal_received, total_interest_received, and total_late_fee_received

5. Modify loan purpose other than in look up to Others

In [ ]:
# Create spark session
import getpass
from pyspark.sql import SparkSession


username = getpass.getuser()
spark = SparkSession.builder.\
    config('spark.shuffle.useOldFetchProtocol','true').\
    config('spark.ui.port', '0').\
    config('spark.sql.warehouse.dir', f'/user/{username}/warehouse').\
    enableHiveSupport().\
    master('yarn').\
    getOrCreate()

In [2]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/user/itv008299/lendingclubproject/raw/loans_repayments_data_csv")

In [3]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_rec_prncp: string (nullable = true)
 |-- total_rec_int: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- total_pymnt: string (nullable = true)
 |-- last_pymnt_amnt: string (nullable = true)
 |-- last_pymnt_d: string (nullable = true)
 |-- next_pymnt_d: string (nullable = true)



In [4]:
loans_repay_schema = """loan_id string, total_principal_received float, 
total_interest_received float, total_late_fee_received float, total_payment_received float,
last_payment_amount float, last_payment_date string, next_payment_date string"""

In [5]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loans_repay_schema) \
.load("/user/itv008299/lendingclubproject/raw/loans_repayments_data_csv")

In [6]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)



## 2. Insert a new column named as ingestion date (current time )

In [7]:
from pyspark.sql.functions import current_timestamp

loans_repay_df_ingestd = loans_repay_raw_df.withColumn("ingest_date", current_timestamp())

In [8]:
loans_repay_df_ingestd

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
68407277,3600.0,821.72,0.0,4421.724,122.67,Jan-2019,null,2023-11-08 17:02:...
68355089,24700.0,979.66,0.0,25679.66,926.35,Jun-2016,null,2023-11-08 17:02:...
68341763,20000.0,2705.92,0.0,22705.924,15813.3,Jun-2017,null,2023-11-08 17:02:...
66310712,19102.35,12361.66,0.0,31464.01,829.9,Feb-2019,Apr-2019,2023-11-08 17:02:...
68476807,10400.0,1340.5,0.0,11740.5,10128.96,Jul-2016,null,2023-11-08 17:02:...
68426831,11950.0,1758.95,0.0,13708.948,7653.56,May-2017,null,2023-11-08 17:02:...
68476668,20000.0,1393.8,0.0,21393.8,15681.05,Nov-2016,null,2023-11-08 17:02:...
67275481,20000.0,1538.51,0.0,21538.51,14618.23,Jan-2017,null,2023-11-08 17:02:...
68466926,10000.0,998.97,0.0,10998.972,1814.48,Aug-2018,null,2023-11-08 17:02:...
68616873,8000.0,939.58,0.0,8939.58,4996.24,Apr-2017,null,2023-11-08 17:02:...


## 3. Drop any nulls

In [9]:
loans_repay_df_ingestd.createOrReplaceTempView("loan_repayments")

In [10]:
# check few columns if null exists
spark.sql("select count(*) from loan_repayments where total_principal_received is null")

count(1)
69


In [11]:
columns_to_check = ["total_principal_received", "total_interest_received", "total_late_fee_received", "total_payment_received", "last_payment_amount"]

In [12]:
loans_repay_filtered_df = loans_repay_df_ingestd.na.drop(subset=columns_to_check)

## 4. Check if total_payment_received is zero, and total_principal_received is not zero. If there is such row then replace the value with sum of total_principal_received, total_interest_received, and total_late_fee_received

In [13]:
loans_repay_filtered_df.createOrReplaceTempView("loan_repayments")

In [15]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0")

count(1)
995


In [14]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0 and total_principal_received > 0.0")

count(1)
46


In [17]:
from pyspark.sql.functions import when, col

loans_payments_fixed_df = loans_repay_filtered_df.withColumn(
   "total_payment_received",
    when(
        (col("total_principal_received") != 0.0) &
        (col("total_payment_received") == 0.0),
        col("total_principal_received") + col("total_interest_received") + col("total_late_fee_received")
    ).otherwise(col("total_payment_received"))
)

In [18]:
loans_payments_fixed_df

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
68407277,3600.0,821.72,0.0,4421.724,122.67,Jan-2019,null,2023-11-08 17:13:...
68355089,24700.0,979.66,0.0,25679.66,926.35,Jun-2016,null,2023-11-08 17:13:...
68341763,20000.0,2705.92,0.0,22705.924,15813.3,Jun-2017,null,2023-11-08 17:13:...
66310712,19102.35,12361.66,0.0,31464.01,829.9,Feb-2019,Apr-2019,2023-11-08 17:13:...
68476807,10400.0,1340.5,0.0,11740.5,10128.96,Jul-2016,null,2023-11-08 17:13:...
68426831,11950.0,1758.95,0.0,13708.948,7653.56,May-2017,null,2023-11-08 17:13:...
68476668,20000.0,1393.8,0.0,21393.8,15681.05,Nov-2016,null,2023-11-08 17:13:...
67275481,20000.0,1538.51,0.0,21538.51,14618.23,Jan-2017,null,2023-11-08 17:13:...
68466926,10000.0,998.97,0.0,10998.972,1814.48,Aug-2018,null,2023-11-08 17:13:...
68616873,8000.0,939.58,0.0,8939.58,4996.24,Apr-2017,null,2023-11-08 17:13:...


## 5.Remove rows with values 0.0 in column total_payment_received. 

These recoreds are not useful

In [20]:
loans_payments_fixed2_df = loans_payments_fixed_df.filter("total_payment_received != 0.0")

## 6. Replace all non suitable values from column last_payment_date, and next_payment_date with null

In [21]:
loans_payments_fixed2_df.filter("last_payment_date = 0.0").count()

48

In [22]:
loans_payments_fixed2_df.filter("next_payment_date = 0.0").count()

24

In [23]:
loans_payments_ldate_fixed_df = loans_payments_fixed2_df.withColumn(
  "last_payment_date",
   when(
       (col("last_payment_date") == 0.0),
       None
       ).otherwise(col("last_payment_date"))
)

In [24]:
loans_payments_ndate_fixed_df = loans_payments_ldate_fixed_df.withColumn(
  "last_payment_date",
   when(
       (col("next_payment_date") == 0.0),
       None
       ).otherwise(col("next_payment_date"))
)

In [25]:
loans_payments_ndate_fixed_df.filter("last_payment_date = 0.0").count()

0

In [26]:
loans_payments_ndate_fixed_df.filter("next_payment_date = 0.0").count()

24

In [27]:
loans_payments_ndate_fixed_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv008299/lendingclubproject/cleaned/loans_repayments_parquet") \
.save()

In [28]:
loans_payments_ndate_fixed_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv008299/lendingclubproject/cleaned/loans_repayments_csv") \
.save()

In [ ]:
spark.stop()